**извлечение признаков из текста на естественном языке**

очистка текста и токенизация

_Евгений Борисов <esborisov@sevsu.ru>_

---

In [1]:
import re
# import numpy as np
# import numpy.random as rng
import pandas as pd
# from tqdm import tqdm
from tqdm.notebook import tqdm

# np.set_printoptions(precision=2) # вывод на печать чисел до 2 знака
pd.options.display.max_colwidth = 200 

tqdm.pandas()

---

Проект Natasha. Набор качественных открытых инструментов для обработки естественного русского языка (NLP).   
https://habr.com/ru/post/516098/

Corus — коллекция русскоязычных NLP-датасетов.   
https://natasha.github.io/corus/

## текст

In [3]:
# загружаем текст
import gzip
with gzip.open('../data/dostoevsky-besy-p2.txt.gz','rt',encoding='utf-8') as f: text = f.read()     
# with gzip.open('../data/lobas-taxisty.txt.gz','rt',encoding='utf-8') as f: data = f.read()     
display(len(text))
print(text[85:1000])

465595

Прошло восемь дней. Теперь, когда уже все прошло, и я пишу хронику, мы уже знаем в чем дело; но тогда мы еще ничего не знали, и естественно, что нам представлялись странными разные вещи. По крайней мере мы со Степаном Трофимовичем в первое время заперлись и с испугом наблюдали издали. Я-то кой-куда еще выходил и по-прежнему приносил ему разные вести, без чего он и пробыть не мог.

Нечего и говорить, что по городу пошли самые разнообразные слухи, то-есть насчет пощечины, обморока Лизаветы Николаевны и прочего случившегося в то воскресенье. Но удивительно нам было то: через кого это все могло так скоро и точно выйти наружу? Ни одно из присутствовавших тогда лиц не имело бы, кажется, ни нужды, ни выгоды нарушить секрет происшедшего. Прислуги тогда не было; один Лебядкин мог бы что-нибудь разболтать, не столько по злобе, потому что вышел тогда в крайнем испуге (а страх к врагу уничтожает и злобу к нему), а


## простой токенайзер

In [5]:
# пробел - разделитель
text.split()

['БЕСЫ',
 'Федор',
 'Михайлович',
 'Достоевский',
 'ЧАСТЬ',
 'ВТОРАЯ',
 'ГЛАВА',
 'ПЕРВАЯ',
 'Ночь',
 'I',
 'Прошло',
 'восемь',
 'дней.',
 'Теперь,',
 'когда',
 'уже',
 'все',
 'прошло,',
 'и',
 'я',
 'пишу',
 'хронику,',
 'мы',
 'уже',
 'знаем',
 'в',
 'чем',
 'дело;',
 'но',
 'тогда',
 'мы',
 'еще',
 'ничего',
 'не',
 'знали,',
 'и',
 'естественно,',
 'что',
 'нам',
 'представлялись',
 'странными',
 'разные',
 'вещи.',
 'По',
 'крайней',
 'мере',
 'мы',
 'со',
 'Степаном',
 'Трофимовичем',
 'в',
 'первое',
 'время',
 'заперлись',
 'и',
 'с',
 'испугом',
 'наблюдали',
 'издали.',
 'Я-то',
 'кой-куда',
 'еще',
 'выходил',
 'и',
 'по-прежнему',
 'приносил',
 'ему',
 'разные',
 'вести,',
 'без',
 'чего',
 'он',
 'и',
 'пробыть',
 'не',
 'мог.',
 'Нечего',
 'и',
 'говорить,',
 'что',
 'по',
 'городу',
 'пошли',
 'самые',
 'разнообразные',
 'слухи,',
 'то-есть',
 'насчет',
 'пощечины,',
 'обморока',
 'Лизаветы',
 'Николаевны',
 'и',
 'прочего',
 'случившегося',
 'в',
 'то',
 'воскресенье.

__*такой метод режет строку только на слова, предложения не разделяет*__

In [7]:
# собираем словарь
words = sorted( set( text.split()) )
display(len(words))
words

22386

['"Bonjour",',
 '"Fils,',
 '"Merci"',
 '"Merci".',
 '"Qu\'un',
 '"Tout',
 '"cette',
 '"qu\'un',
 '"В',
 '"Ваше',
 '"Вашего',
 '"Ведь',
 '"Вот,',
 '"Вы',
 '"Господи!"',
 '"Да',
 '"Днем',
 '"Думы"',
 '"Если',
 '"Еще',
 '"Еще,',
 '"Знаете',
 '"Знай,',
 '"Ибо',
 '"Искание',
 '"Как-нибудь',
 '"Лембка",',
 '"Лембку"',
 '"Лембку",',
 '"Лембку".',
 '"Либерал',
 '"Милостивые',
 '"Мы',
 '"Мысль',
 '"Мя,',
 '"На',
 '"Наплевать',
 '"Нашего"',
 '"Не',
 '"Несчастные"',
 '"Нет,',
 '"Неужели',
 '"Но...',
 '"Нож,',
 '"Ночью',
 '"Ну',
 '"Ну,',
 '"Нужно',
 '"Однако',
 '"По',
 '"Полон',
 '"Поскорей,',
 '"Правда,',
 '"Припадает',
 '"Пугает',
 '"С',
 '"Светлую',
 '"Соловья',
 '"Сто',
 '"Такой',
 '"Ты,',
 '"Угрюмые',
 '"Успеешь,',
 '"Франко-прусская',
 '"Хитрый"',
 '"Чти',
 '"Что',
 '"Шатова',
 '"Шигалевщина",',
 '"Шпигулинские!"',
 '"Шпигулинские".',
 '"Экой',
 '"Это',
 '"Этот',
 '"Я',
 '"а',
 '"а,',
 '"атеист',
 '"бездарности";',
 '"беспардонный',
 '"бесчисленными',
 '"благородным"',
 '"богоносец"',
 '"бог

## NLTK токенайзер

In [8]:
# NLTK токенайзер
from nltk.tokenize import sent_tokenize as nltk_sentence_split
from nltk.tokenize import word_tokenize as nltk_tokenize_word

text_ = [ 
    nltk_tokenize_word(s,language='russian') # разбиваем предложения на слова
    for s in nltk_sentence_split(text,language='russian') # режем текст на отдельные предложения
]

display( len(text_) )
text_

5495

[['БЕСЫ',
  'Федор',
  'Михайлович',
  'Достоевский',
  'ЧАСТЬ',
  'ВТОРАЯ',
  'ГЛАВА',
  'ПЕРВАЯ',
  'Ночь',
  'I',
  'Прошло',
  'восемь',
  'дней',
  '.'],
 ['Теперь',
  ',',
  'когда',
  'уже',
  'все',
  'прошло',
  ',',
  'и',
  'я',
  'пишу',
  'хронику',
  ',',
  'мы',
  'уже',
  'знаем',
  'в',
  'чем',
  'дело',
  ';',
  'но',
  'тогда',
  'мы',
  'еще',
  'ничего',
  'не',
  'знали',
  ',',
  'и',
  'естественно',
  ',',
  'что',
  'нам',
  'представлялись',
  'странными',
  'разные',
  'вещи',
  '.'],
 ['По',
  'крайней',
  'мере',
  'мы',
  'со',
  'Степаном',
  'Трофимовичем',
  'в',
  'первое',
  'время',
  'заперлись',
  'и',
  'с',
  'испугом',
  'наблюдали',
  'издали',
  '.'],
 ['Я-то',
  'кой-куда',
  'еще',
  'выходил',
  'и',
  'по-прежнему',
  'приносил',
  'ему',
  'разные',
  'вести',
  ',',
  'без',
  'чего',
  'он',
  'и',
  'пробыть',
  'не',
  'мог',
  '.'],
 ['Нечего',
  'и',
  'говорить',
  ',',
  'что',
  'по',
  'городу',
  'пошли',
  'самые',
  'разноо

In [9]:
# собираем словарь
words = sorted( set.union( *[ set(s) for s in text_ ] ) )
display(len(words))
words

16658

['!',
 "'",
 "''",
 '(',
 ')',
 ',',
 '-',
 '.',
 '..',
 '...',
 '187',
 '19-го',
 ':',
 ';',
 '?',
 'Adieu',
 'Alea',
 'Allons',
 'Augustin',
 'Avis',
 'Ax',
 'Bonjour',
 "C'est",
 "C'йtait",
 'Candide',
 'Cela',
 'Cette',
 'Cher',
 'Chere',
 'Cozak',
 'Don',
 'Elle',
 'Eloignez-la',
 'Enfin',
 'Et',
 'Excellent',
 'Fantichambre',
 'Fautre',
 'Fils',
 'He',
 'I',
 'II',
 'III',
 'IV',
 'Il',
 'Ils',
 'Incognito',
 'Internationale',
 "J'ai",
 "J'йtais",
 'Je',
 'Karmazinoff',
 'Lembke',
 'Littre',
 'Maman',
 'Mein',
 'Merci',
 'Nicolas',
 'Oui',
 'Pardon',
 'Pas',
 'Passons',
 'Pйtersbourg',
 "Qu'en",
 "Qu'un",
 'Quelque',
 'Rosenthal',
 'Savez',
 'Savez-vous',
 'Se',
 'Tant',
 'Teliatnikoff',
 'To-есть',
 'Tous',
 'Tout',
 'V',
 'VI',
 'VII',
 'Voilа',
 'Voltaire',
 'Votre',
 'Vous',
 'Voyez-vous',
 '``',
 'a',
 'abreuve',
 'activitй',
 'ami',
 'amie',
 'amis',
 'annйe',
 'ans',
 'arrкter',
 'au',
 "aujourd'hui",
 'autour',
 'avait',
 'avec',
 'bas',
 'battre',
 'beaucoup',
 'belle',


количество слов 16 658    
против 22 386 в случае "простого" разделения по пробелу    

NLTK обеспечивает разделение на отдельные предложения   
и более качественную токенизацию   
разделяя конструкции типа '(Послышался'   

## токенайзер для русского языка из пакета  Natasha 

In [13]:
from razdel import sentenize
from razdel import tokenize

text_ = [ [ t.text for t in tokenize(s.text) ] for s in sentenize(text) ]
text_

[['БЕСЫ',
  'Федор',
  'Михайлович',
  'Достоевский',
  'ЧАСТЬ',
  'ВТОРАЯ',
  'ГЛАВА',
  'ПЕРВАЯ',
  'Ночь',
  'I',
  'Прошло',
  'восемь',
  'дней',
  '.'],
 ['Теперь',
  ',',
  'когда',
  'уже',
  'все',
  'прошло',
  ',',
  'и',
  'я',
  'пишу',
  'хронику',
  ',',
  'мы',
  'уже',
  'знаем',
  'в',
  'чем',
  'дело',
  ';',
  'но',
  'тогда',
  'мы',
  'еще',
  'ничего',
  'не',
  'знали',
  ',',
  'и',
  'естественно',
  ',',
  'что',
  'нам',
  'представлялись',
  'странными',
  'разные',
  'вещи',
  '.'],
 ['По',
  'крайней',
  'мере',
  'мы',
  'со',
  'Степаном',
  'Трофимовичем',
  'в',
  'первое',
  'время',
  'заперлись',
  'и',
  'с',
  'испугом',
  'наблюдали',
  'издали',
  '.'],
 ['Я-то',
  'кой-куда',
  'еще',
  'выходил',
  'и',
  'по-прежнему',
  'приносил',
  'ему',
  'разные',
  'вести',
  ',',
  'без',
  'чего',
  'он',
  'и',
  'пробыть',
  'не',
  'мог',
  '.'],
 ['Нечего',
  'и',
  'говорить',
  ',',
  'что',
  'по',
  'городу',
  'пошли',
  'самые',
  'разноо

In [14]:
# собираем словарь
words = sorted( set.union( *[ set(s) for s in text_ ] ) )
display(len(words))
words

16643

['!',
 '!!',
 '!!!..',
 '!..',
 '!?',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '..',
 '...',
 '187',
 '19-го',
 ':',
 ';',
 '?',
 '?..',
 'Adieu',
 'Alea',
 'Allons',
 'Augustin',
 'Avis',
 'Ax',
 'Bonjour',
 'C',
 'Candide',
 'Cela',
 'Cette',
 'Cher',
 'Chere',
 'Cozak',
 'Don',
 'Elle',
 'Eloignez-la',
 'Enfin',
 'Et',
 'Excellent',
 'Fantichambre',
 'Fautre',
 'Fils',
 'He',
 'I',
 'II',
 'III',
 'IV',
 'Il',
 'Ils',
 'Incognito',
 'Internationale',
 'J',
 'Je',
 'Karmazinoff',
 'Lembke',
 'Littre',
 'Maman',
 'Mein',
 'Merci',
 'Nicolas',
 'Oui',
 'P',
 'Pardon',
 'Pas',
 'Passons',
 'Qu',
 'Quelque',
 'Rosenthal',
 'Savez',
 'Savez-vous',
 'Se',
 'Tant',
 'Teliatnikoff',
 'To-есть',
 'Tous',
 'Tout',
 'V',
 'VI',
 'VII',
 'Voil',
 'Voltaire',
 'Votre',
 'Vous',
 'Voyez-vous',
 'a',
 'abreuve',
 'activit',
 'ai',
 'allemand',
 'ami',
 'amie',
 'amis',
 'ann',
 'ans',
 'appelle',
 'arr',
 'assure',
 'au',
 'aujourd',
 'autour',
 'avait',
 'avec',
 'aveugle',
 'b',
 'bas',
 'battr

## стеминг  для русского языка из пакета  NLTK

In [18]:
from nltk.tokenize import sent_tokenize as nltk_sentence_split
from nltk.tokenize import word_tokenize as nltk_tokenize_word
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('russian')

text_ = [ 
    [ # разбиваем предложения на слова
        stemmer.stem(t) # выполняем стеминг
        for t in nltk_tokenize_word(s,language='russian') 
    ] # разбиваем предложения на слова
    for s in nltk_sentence_split(text,language='russian') # режем текст на отдельные предложения
]

display( len(text_) )
text_

5495

[['бес',
  'федор',
  'михайлович',
  'достоевск',
  'част',
  'втор',
  'глав',
  'перв',
  'ноч',
  'I',
  'прошл',
  'восем',
  'дне',
  '.'],
 ['тепер',
  ',',
  'когд',
  'уж',
  'все',
  'прошл',
  ',',
  'и',
  'я',
  'пиш',
  'хроник',
  ',',
  'мы',
  'уж',
  'зна',
  'в',
  'чем',
  'дел',
  ';',
  'но',
  'тогд',
  'мы',
  'ещ',
  'нич',
  'не',
  'знал',
  ',',
  'и',
  'естествен',
  ',',
  'что',
  'нам',
  'представля',
  'стран',
  'разн',
  'вещ',
  '.'],
 ['по',
  'крайн',
  'мер',
  'мы',
  'со',
  'степан',
  'трофимович',
  'в',
  'перв',
  'врем',
  'заперл',
  'и',
  'с',
  'испуг',
  'наблюда',
  'изда',
  '.'],
 ['я-т',
  'кой-куд',
  'ещ',
  'выход',
  'и',
  'по-прежн',
  'принос',
  'ем',
  'разн',
  'вест',
  ',',
  'без',
  'чег',
  'он',
  'и',
  'проб',
  'не',
  'мог',
  '.'],
 ['неч',
  'и',
  'говор',
  ',',
  'что',
  'по',
  'город',
  'пошл',
  'сам',
  'разнообразн',
  'слух',
  ',',
  'то-ест',
  'насчет',
  'пощечин',
  ',',
  'обморок',
  'лиза

In [19]:
# собираем словарь
words = sorted( set.union( *[ set(s) for s in text_ ] ) )
display(len(words))
words

8275

['!',
 "'",
 "''",
 '(',
 ')',
 ',',
 '-',
 '.',
 '..',
 '...',
 '187',
 '19-го',
 ':',
 ';',
 '?',
 'Adieu',
 'Alea',
 'Allons',
 'Augustin',
 'Avis',
 'Ax',
 'Bonjour',
 "C'est",
 'Candide',
 'Cela',
 'Cette',
 'Cher',
 'Chere',
 'Cozak',
 'Cьйта',
 'Don',
 'Elle',
 'Eloignez-la',
 'Enfin',
 'Et',
 'Excellent',
 'Fantichambre',
 'Fautre',
 'Fils',
 'He',
 'I',
 'II',
 'III',
 'IV',
 'Il',
 'Ils',
 'Incognito',
 'Internationale',
 "J'ai",
 'Je',
 'Jьйтаис',
 'Karmazinoff',
 'Lembke',
 'Littre',
 'Maman',
 'Mein',
 'Merci',
 'Nicolas',
 'Oui',
 'Pardon',
 'Pas',
 'Passons',
 'Pйтерсбоург',
 "Qu'en",
 "Qu'un",
 'Quelque',
 'Rosenthal',
 'Savez',
 'Savez-vous',
 'Se',
 'Tant',
 'Teliatnikoff',
 'Tous',
 'Tout',
 'Tо-ест',
 'V',
 'VI',
 'VII',
 'Voltaire',
 'Votre',
 'Vous',
 'Voyez-vous',
 'Vо',
 '``',
 'a',
 'abreuve',
 'ami',
 'amie',
 'amis',
 'ans',
 'au',
 "aujourd'hui",
 'autour',
 'avait',
 'avec',
 'bas',
 'battre',
 'beaucoup',
 'belle',
 'belles',
 'bien',
 'bienfaisant',
 'bon

количество слов в словаре уменьшилось   
8275 против 16 658 случае NLTK токенайзера без стеминга

## лемматизатор для русского языка из пакета Natasha

In [33]:
from natasha import Doc
from natasha import Segmenter
from natasha import MorphVocab
from natasha import NewsEmbedding
from natasha import NewsMorphTagger

# выполняем сегментацию строки на токены
doc = Doc(text) 
doc.segment( Segmenter() )
# doc.sents[77].tokens
# выполняем морфологический анализ
doc.tag_morph( NewsMorphTagger( NewsEmbedding() ) )

# выполняем лематизацию
morph_vocab = MorphVocab() # лемматизатор
for t in doc.tokens: t.lemmatize(morph_vocab)

# собираем нормализованный текст 
text_ =  [ [ t.lemma for t in s.tokens ] for s in doc.sents ]

In [34]:
text_

[['бес',
  'федор',
  'михаилович',
  'достоевский',
  'часть',
  'второй',
  'глава',
  'первый',
  'ночь',
  'i',
  'пройти',
  'восемь',
  'день',
  '.'],
 ['теперь',
  ',',
  'когда',
  'уже',
  'весь',
  'пройти',
  ',',
  'и',
  'я',
  'писать',
  'хроника',
  ',',
  'мы',
  'уже',
  'знать',
  'в',
  'что',
  'дело',
  ';',
  'но',
  'тогда',
  'мы',
  'еще',
  'ничто',
  'не',
  'знать',
  ',',
  'и',
  'естественно',
  ',',
  'что',
  'мы',
  'представляться',
  'странный',
  'разный',
  'вещь',
  '.'],
 ['по',
  'крайний',
  'мера',
  'мы',
  'с',
  'степан',
  'трофимович',
  'в',
  'первый',
  'время',
  'запереться',
  'и',
  'с',
  'испуг',
  'наблюдать',
  'издать',
  '.'],
 ['я-то',
  'кой-куда',
  'еще',
  'выходить',
  'и',
  'по-прежнему',
  'приносить',
  'он',
  'разный',
  'весть',
  ',',
  'без',
  'что',
  'он',
  'и',
  'пробыть',
  'не',
  'мочь',
  '.'],
 ['нечего',
  'и',
  'говорить',
  ',',
  'что',
  'по',
  'город',
  'пойти',
  'самый',
  'разнообразный

In [35]:
# собираем словарь
words = sorted( set.union( *[ set(s) for s in text_ ] ) )
display(len(words))
words

8557

['!',
 '!!',
 '!!!..',
 '!..',
 '!?',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '..',
 '...',
 '187',
 '19-й',
 ':',
 ';',
 '?',
 '?..',
 'a',
 'abreuve',
 'activit',
 'adieu',
 'ai',
 'alea',
 'allemand',
 'allons',
 'ami',
 'amie',
 'amis',
 'ann',
 'ans',
 'appelle',
 'arr',
 'assure',
 'au',
 'augustin',
 'aujourd',
 'autour',
 'avait',
 'avec',
 'aveugle',
 'avis',
 'ax',
 'b',
 'bas',
 'battre',
 'bauches',
 'beaucoup',
 'belle',
 'belles',
 'bien',
 'bienfaisant',
 'bon',
 'bonjour',
 'boudoir',
 'bourru',
 'brisons-l',
 'bruit',
 'c',
 'cacha',
 'calme',
 'candide',
 'carri',
 'ce',
 'cela',
 'ces',
 'cette',
 'ch',
 'chambre',
 'chaque',
 'charmant',
 'charmante',
 'cher',
 'chere',
 'chet',
 'chez',
 'chose',
 'choses',
 'choses-l',
 'cochon',
 'comme',
 'commencerai',
 'comment',
 'comparaison',
 'comprenez',
 'compris',
 'connais',
 'connaissez',
 'consenti',
 'content',
 'convertir',
 'cozak',
 'critiques',
 'croire',
 'croyez',
 'croyez-vous',
 'd',
 'dans',
 'date',
 'd